In [8]:
import os
import random
import shutil

import numpy as np

In [12]:
IS_RESET = True
SRC_DATA = './sampled_data_1184'
DST_DATA = './federated_data_5_balanced_iid'
CLIENTS = 6

In [13]:
def get_files(root_dir, ext=('.jpg', '.png')):
    queue = [root_dir]
    while len(queue) != 0:
        nest_dir = queue.pop()
        label = None
        files = list()
        with os.scandir(nest_dir) as it:
            for entry in it:
                if not entry.name.startswith('.') and entry.is_file():
                    if entry.name.endswith(ext):
                        label = os.path.basename(os.path.dirname(entry.path)) # dirname is label
                        files.append(entry.path)
                elif not entry.name.startswith('.') and entry.is_dir():
                    queue.append(entry.path)
            if label is not None:
                yield label, files

In [14]:
data_root = os.path.abspath(os.path.expanduser(SRC_DATA))
dst_base = os.path.abspath(os.path.expanduser(DST_DATA))

if IS_RESET:
    if os.path.exists(dst_base):
        shutil.rmtree(dst_base)

for label, files in get_files(data_root):
    files_list = np.array_split(files, CLIENTS)
    for i in range(0, CLIENTS):
        dst = os.path.join(dst_base, str(i), label)
        os.makedirs(dst, exist_ok=True) 
        for path in files_list[i]:
            shutil.copy(path, dst)